# <p align="center"> AGRON 935 - Semester project</p>

**Name:** Javier Fernandez <br/>
**Semester:** Spring 2019 <br/>
**Project area:** Agronomy <br/>

## Table of contents
1. [Motivation for the project](#motivation_project)
2. [Current state and roadblocks](#state_roadblocks)
3. [Future steps](#future_steps)

<a name="motivation_project"></a>
### 1. Motivation for the project

The objective is to create a code that can determine and calculate the HTT (Hourly Thermal Time) and DTT (Daily Thermal Time) using Kansas Mesonet database for grain filling samples. Periods of time for HTT and DTT calculations will be between a flowering date (starting point) and a sampling date for each data point.

Moreover, the code will allow the user to choose a minimum and maximum cardinal temperatures for calculations on HTT and DTT. This will be useful to evaluate and compare different base temperatures for grain filling.

<p align="center"><img src="project_sketch.jpg" alt="sketch_image" width="600"/></p>

<a name="state_roadblocks"></a>
### 2. Current state and Roadblocks

The code now is calculating DTT for each sample in the data. Still the code is needing the input weather data as a .csv file directly from Kansas Mesonet website, and then is organizing the data to obtain each sample DTT.

##### Steps:
1. [Import modules](#import_modules)
2. [Import data](#import_data)
3. [Handling missing values](#missing_values)
4. [Calculate thermal time for each day](#daily_TT)
5. [Sumation of thermal time for each sample](#sample_TT)

Input files neccesary are:

  - daily and hourly weather .csv file directly from Kansas Mesonet website
  
  - grain filling or biomass data, especially having columns for sampling date (here "/Samp..Date2") and starting counting date (here /Silking2)

<a name="import_modules"></a>
***2.1 Import modules***

In [ ]:
import pandas as pd
import numpy as np

# Working directory for the project files
dirname = 'C:/Users/jafernandez.USERS/Desktop/Coding/sandbox/project/'

<a name="import_data"></a>
***2.2 Import data***

Grain filling data, with sample and flowering dates:

In [ ]:
df = pd.read_csv(dirname + '2017_GF.csv') # N, H, WC, Strip, and Time are values representing treatments and experimental design
df = pd.DataFrame(df)
df.head()

Daily weather data:

In [ ]:
file = open(dirname + "Daily_2017.csv", "r") 
wtrD = pd.read_csv(file, sep = ",")  # We use this method of importing datatframes as I was encountering issues beacuse of multiple headers
file.close() 

#Renaming and dropping unused columns
wtrD = wtrD.rename(columns={'Timestamp': 'Day', 'AirTemperature': 'Max', 'AirTemperature.1': 'Min'})
wtrD = wtrD.drop([0,1], axis=0) # remove two first rows
wtrD = wtrD.iloc[:,[0,2,3]] # keep only selected columns

# Converting values to type = float. Errors = coerce return invalid conversions to Nan
wtrD['Min'] = pd.to_numeric(wtrD['Min'], errors='coerce')
wtrD['Max'] = pd.to_numeric(wtrD['Max'], errors='coerce')

# Calculating daily mean temperatures
wtrD['Average'] = ((wtrD['Min']) + (wtrD['Max']))/2

#wtrD.loc[wtrD['Average'] < 10, 'C'] = 0
#wtrD.loc[wtrD['Average'] > 10, 'C'] = wtrD['Average']

wtrD.head()

Hourly weather data:

In [ ]:
# For the hourly data, Mesonet does not allow to obtain more than 30 days in one same file. In this case we are working with a range of 
# 3 months, so we will have a total of 3 files

wtrH = pd.DataFrame(columns = ['Day', 'Temp']) # Creating an empty dataframe

# For loop to open three months at the same time and appending them at the end
for i in ['jul', 'aug', 'sept']:
    if i == 'jul':
        file = "july_hourly.csv"
    elif i == 'aug':
        file = "aug_hourly.csv"
    elif i == 'sept':
        file = "sept_hourly.csv"
        
    daf = open(dirname + file, "r") 
    dat = pd.read_csv(daf, sep = ",")
    daf.close() 

    dat = dat.rename(columns={'Timestamp': 'Day', 'AirTemperature': 'Temp'})
    dat = dat.drop([0,1], axis=0)
    dat = dat.iloc[:,[0,2]]
    
    wtrH = wtrH.append(dat)

# Converting values to type = float. Errors = coerce return invalid conversions to Nan
wtrH['Temp'] = pd.to_numeric(wtrH['Temp'], errors='coerce') 


# As the day column contains the day and the hour, we need to separate in two columns these values
# For this we split that column based on the space between strings
new = wtrH["Day"].str.split(" ", expand = True) 
 
wtrH.drop(columns =["Day"])  # Drop the old "Day" column

wtrH["Day"]= new[0]
wtrH["Time"]= new[1]

wtrH.head().append(wtrH.tail())  # Print the first five and last five rows to check that the data is OK

<a name="missing_values"></a>
***2.3 Handling missing values***

In [ ]:
# Now, let's check missing values in our dataframes
#print(df.isna().sum())
#print(wtrH.isna().sum())
#print(wtrD.isna().sum())

*This was the most difficult part to define, and I still have not decide if this is the best way to do it. For now, I was able to solve it by:*

- For the grain filling database, if missing values are encountered (in this example, is 1 row) we will just drop that row or measurement

- For the weather database, missing temperatures will be estimated with the previous recorded value. In this case, we do not have missing values in the daily weather file, so we will focus on the hourly weather data.

In [ ]:
df = df[pd.notnull(df['DW_mg'])] # Drop rows that are not null in the DW_mg column

wtrH = wtrH.fillna(method='ffill')  # ffill stands for forward fill, so it is using the last notNan value to fill a Nan

<a name="daily_TT"></a>
***2.4 Calculate thermal time for each day***

We will use McMaster and Wilhelm (1997) method for calculating DTT, which has been widely used in studies. It assumes a linear relationship using the mean daily temperature, with an upper critical value.

- Tmax is the maximum temperature, Tmin is the minimum temperature, Tavg = (Tmax + Tmin)/2 
- Tbase is the base temperature = 8 C
- Tupp is the upper threshold temperature = 40 C

Input = Tmin and Tmax (daily)

In [ ]:
# We define a function to calculate each GD for a given temperature.
def GDD1 (T_avg, tbase=8, Tupp = 40):
    if T_avg <= tbase:
        GD = 0
    elif T_avg >= Tupp:
        GD = Tupp - tbase
    else:
        GD = T_avg - tbase
    
    return GD

In [ ]:
# We apply the function to each row in the daily weather file and we store it into a new column named GDD1.
wtrD['GDD1'] = wtrD.Average.apply(GDD1)
wtrD.head()

<a name="sample_TT"></a>
***2.5 Sumation of thermal time for each sample***

In [ ]:
# Set the dates as a same format for all the values we will be using
df['Sampling'] = pd.to_datetime(df['Sampling'], format='%m/%d/%Y')
df['Flowering'] = pd.to_datetime(df['Flowering'], format='%m/%d/%Y')
wtrD['Day'] = pd.to_datetime(wtrD['Day'], format='%Y/%m/%d')
wtrH['Day'] = pd.to_datetime(wtrH['Day'], format='%Y/%m/%d')

# In this section we calculated the sum of GDD or thermal time based on the new calculated weather file.
# For this, we define a function that actually performs the sumation indexing on sampling and flowering dates

def calc (start, end):
      
    TT = wtrD[(wtrD.Day >= start) & (wtrD.Day <= end)].GDD1.sum()
    return TT

In [ ]:
# Then we need to iterate this function across all my rows in the dataframe, so I found the intertuples function useful.

mylist = [] # create an empty list
for row in df.itertuples():
    TT = calc(row.Flowering, row.Sampling)
    mylist.append(TT)   #append the TT values for each row in a list

# Add that list as a new column named GDD1 in our dataframe

df['GDD1'] = np.asarray(mylist)
df.head()

<a name="future_steps"></a>
### 3. Future Steps

- The following step will be to create a function that caluclates HTT using the hourly datafile, and then calculate the sumation correspondent to each sample date. Basically, it will do the same as DTT but using hourly data.

- The next part of the code should be able to retrieve the specific .csv file from Mesonet website, just entering the dates and weather station.